In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('/code/competition1/input/train.csv')
test = pd.read_csv('/code/competition1/input/test.csv')
gender_submission = pd.read_csv('/code/competition1/gender_submission.csv')

In [3]:
"""trainとtestでデータの結合"""
df_all = pd.concat([train, test], axis=0)
df_all = df_all.reset_index(drop=True)
all_family = df_all.sort_values('Name')

In [4]:
"""名前の被りを調べる""" 
names = defaultdict(int)
one = 0
two = 0
reigai = 0
for fid in all_family.index:
    name = all_family.loc[fid, 'Name']
    spi_name = name.split()
    if spi_name[0][-1] == ',':
        names[spi_name[0]] += 1
        one += 1
    elif spi_name[1][-1]:
        names[spi_name[0]+' '+spi_name[1]] += 1
        two += 1
    else:
        reigai += 1
# print('one: ', one, 'two: ', two, 'reigai: ', reigai)

In [5]:
"""２以上の被りが合った名前リストを作成"""
# some_names = {}
names_list = []
for name in names.items():
    if name[1] > 1:
#         some_names[name[0]] = []
        names_list.append(name[0])
# print(some_names, len(some_names), sep='\n')

In [6]:
"""被り名前をグループ単位で列挙"""
# for i, name in enumerate(df_all.loc[:, 'Name']):
#     nasp = name.split()
#     if nasp[0] in names_list:
#         some_names[nasp[0]].append(name)
#     elif (nasp[0] + ' ' +nasp[1]) in names_list:
#         some_names[nasp[0] + ' ' +nasp[1]].append(name)
# display(some_names)

'被り名前をグループ単位で列挙'

In [7]:
"""名前被りのグループでラベルをつける"""
survived_list = []
died_list = []
for i in names_list:
    data = df_all[df_all['Name'].str.startswith(i)]
    train_data = data[data.loc[:, 'PassengerId'] < 892]
    test_data = data[data.loc[:, 'PassengerId'] > 891]
    data_fe = train_data[train_data.loc[:, 'Sex'] == 'female']
    data_ma = train_data[(train_data['Name'].str.contains('Master')) & (train_data.loc[:, 'Sex'] == 'male')]
    felen = len(data_fe)
    malen = len(data_ma)
    flag = 0
    flag_mabo = 0
    
    data_maboroshi = train_data[(train_data.loc[:, 'Sex'] == 'male') & (train_data.loc[:, 'Age'] > 15)]
    mabolen = len(data_maboroshi)
    if mabolen > 0:
        sulen = len(data_maboroshi[data_maboroshi.loc[:, 'Survived'] == 1])
        if sulen == mabolen:
            flag_mabo = 1
    
    if felen > 0:
        sulen = len(data_fe[data_fe.loc[:, 'Survived'] == 1])
        if sulen == felen:
            flag = 1
            if malen > 0:
                sulen = len(data_ma[data_ma.loc[:, 'Survived'] == 1])
                if sulen != malen:
                    flag = 2
        else:
            flag = 2
    elif malen > 0:
        sulen = len(data_ma[data_ma.loc[:, 'Survived'] == 1])
        if sulen == malen:
            flag = 1
        else:
            flag = 2        
    
    if flag == 1:
        if flag_mabo == 1:
            for li in data.loc[:, 'PassengerId']:
                survived_list.append(li)
        survived_data = data[(data.loc[:, 'Sex'] == 'female') | (data['Name'].str.contains('Master'))]
        for li in survived_data.loc[:, 'PassengerId']:
            survived_list.append(li)
    elif flag == 2:
        for li in data.loc[:, 'PassengerId']:
            died_list.append(li)
    elif felen == 0 and malen == 0 and flag_mabo == 1:
        for li in data.loc[:, 'PassengerId']:
            survived_list.append(li)

In [30]:
survived_list = sorted(list(set(survived_list)))
died_list = sorted(list(set(died_list)))
# print(len(survived_list), len(died_list), len(survived_list) + len(died_list))

In [9]:
s_list = survived_list.copy()
d_list = died_list.copy()
s_list.extend(d_list)
s_list.sort()
# print(s_list)
# print(len(s_list))
# len(pd.Series(s_list).unique())

In [10]:
all_status_dict = {}
for i in range(1, 1310):
    if i in survived_list:
        all_status_dict[i] = "S"
    elif i in died_list:
        all_status_dict[i] = "D"
    else:
        all_status_dict[i] = "N"

In [11]:
family_status = pd.DataFrame(pd.Series(all_status_dict)).reset_index()
family_status = family_status.rename(columns={'index':'PassengerId', 0:'fstatus'})
# display(family_status)

In [12]:
df_all_status = pd.merge(df_all, family_status)
# display(df_all_status)

In [13]:
ticket_num = pd.DataFrame(df_all.Ticket.value_counts()).reset_index().rename(columns={'Ticket':'Ticket_Num', 'index':'Ticket'})
# display(ticket_num)

In [14]:
df_all_status = pd.merge(df_all_status, ticket_num, on='Ticket')
# df_all_status

In [15]:
train = df_all_status[df_all_status.loc[:, 'PassengerId'] < 892].sort_values('PassengerId').reset_index(drop=True)
test = df_all_status[df_all_status.loc[:, 'PassengerId'] > 891].sort_values('PassengerId').reset_index(drop=True)
# display(train, test)

In [16]:
train['Embarked'] = train['Embarked'].fillna('S')
test['Embarked'] = test['Embarked'].fillna('S')

train['family'] = train['SibSp'] + train['Parch']
test['family'] = test['SibSp'] + test['Parch']

In [17]:
def add_dummy(df):
    df['Pclass'] = df['Pclass'].astype(np.str)
    temp = pd.get_dummies(df[['Sex','Embarked','Pclass', 'fstatus']], drop_first = False)
    temp['PassengerId'] = df['PassengerId']
    return pd.merge(df, temp)

In [18]:
train = add_dummy(train)
test = add_dummy(test)

In [19]:
# display(train.head(), test.head())

In [20]:
# display(train.columns)

In [21]:
#Drop unnecessary feature
def get_feature_mat(df):
    temp = df.drop(columns=['PassengerId','Name','Sex','SibSp','Parch','Ticket','Embarked','Age','Cabin','Pclass','fstatus'])
    try:
        temp = temp.drop(columns=['Survived'])
    except:
        pass
    print (temp)
    return temp

In [22]:
x_train = get_feature_mat(train)
x_test = get_feature_mat(test)

        Fare  Ticket_Num  family  Sex_female  Sex_male  Embarked_C  \
0     7.2500           1       1           0         1           0   
1    71.2833           2       1           1         0           1   
2     7.9250           1       0           1         0           0   
3    53.1000           2       1           1         0           0   
4     8.0500           1       0           0         1           0   
..       ...         ...     ...         ...       ...         ...   
886  13.0000           1       0           0         1           0   
887  30.0000           1       0           1         0           0   
888  23.4500           4       3           1         0           0   
889  30.0000           1       0           0         1           1   
890   7.7500           1       0           0         1           0   

     Embarked_Q  Embarked_S  Pclass_1  Pclass_2  Pclass_3  fstatus_D  \
0             0           1         0         0         1          0   
1             0

In [23]:
y_train = train['Survived'].astype(int)
# display(y_train)

In [24]:
# display(x_train,x_test)

In [25]:
xgb = XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=80)
xgb.fit(x_train, y_train)
y_test = xgb.predict(x_test)
y_test

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [26]:
test['Survived'] = y_test.astype(np.int)

In [27]:
submission = test[["PassengerId","Survived"]]
# display(submission)

In [28]:
submission.to_csv('../submission_hirochon.csv', index=False)